# News Classifier 


### this is a joint affair of Armin Behjati and Bahram Mohammadpour

In this notebook we try to make a news classifier based on the news language model we made earlier . back then we saved the encoder to use it here as the backbone of this model . 

as before we use the fantastic fastai library to make things faster and easier !

In [1]:
import warnings
warnings.filterwarnings("ignore")
from fastai.text import *
import html

/home/bahram/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
BOS = 'xbos'  
FLD = 'xfld'  

PATH=Path('data/')

In [3]:
LM_PATH=Path('data/news_lm/')

In [4]:
CLAS_PATH=Path('data/news_clas/')
CLAS_PATH.mkdir(exist_ok=True)

In [5]:
df = pd.read_pickle('news_out')

In [6]:
df.head()

,text,label
0,,5
1,معاون بازاريابي فروش ايران خودرو با بيان اين ...,5
2,,5
3,,5
4,به گزارش روابط عمومی واموربین الملل سازمان منط...,5


In [7]:
df['text'].replace('', np.nan, inplace=True)

In [8]:
df.head()

,text,label
0,NaN,5
1,معاون بازاريابي فروش ايران خودرو با بيان اين ...,5
2,NaN,5
3,NaN,5
4,به گزارش روابط عمومی واموربین الملل سازمان منط...,5


In [9]:
df.dropna(subset=['text'], inplace=True)

In [10]:
df.shape

(10171, 2)

In [11]:
df.head()

,text,label
1,معاون بازاريابي فروش ايران خودرو با بيان اين ...,5
4,به گزارش روابط عمومی واموربین الملل سازمان منط...,5
5,بانک رفاه در راستای سیاست ها و تصمیمات ارزی مت...,5
6,مديرعامل سازمان منطقه آزاد كيش توجه به ارتقاء ...,5
8,مدیرعامل بیمه ایران می‌گوید درباره نحوه پرداخت...,5


In [12]:
cols = df.columns.tolist()
cols

['text', 'label']

In [13]:
cols = cols[-1:] + cols[:-1]
cols

['label', 'text']

In [14]:
df = df[cols]
df.head()

,label,text
1,5,معاون بازاريابي فروش ايران خودرو با بيان اين ...
4,5,به گزارش روابط عمومی واموربین الملل سازمان منط...
5,5,بانک رفاه در راستای سیاست ها و تصمیمات ارزی مت...
6,5,مديرعامل سازمان منطقه آزاد كيش توجه به ارتقاء ...
8,5,مدیرعامل بیمه ایران می‌گوید درباره نحوه پرداخت...


In [15]:
df.rename(columns = {'label':'labels'}, inplace = True)
df.head()

,labels,text
1,5,معاون بازاريابي فروش ايران خودرو با بيان اين ...
4,5,به گزارش روابط عمومی واموربین الملل سازمان منط...
5,5,بانک رفاه در راستای سیاست ها و تصمیمات ارزی مت...
6,5,مديرعامل سازمان منطقه آزاد كيش توجه به ارتقاء ...
8,5,مدیرعامل بیمه ایران می‌گوید درباره نحوه پرداخت...


In [18]:
service_ids = list(set(df["labels"]))
service_idtoid = {service_ids[i]:i for i in range(len(service_ids))}

In [19]:
service_idtoid

{10: 0, 2: 1, 18: 2, 5: 3}

In [40]:
lb = df.as_matrix(columns=df.columns[0:])

/home/bahram/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [43]:
len(lb[0])

2

In [44]:
nl = lb[:,0]

In [45]:
nl

array([2, 2, 18, ..., 18, 2, 5], dtype=object)

In [46]:
tmp_labels = list()
for i in nl:
    tmp_labels.append(service_idtoid[i])

In [50]:
df['labels'] = tmp_labels

In [51]:
df.head()

,labels,text
2352,1,| سخنگوی وزارت امور خارجه گفت: جمهوری اسلامی ...
1752,1,امام جمعه قم گفت: مسئله حجاب رکن دفاع از حقوق ...
20103,2,| جوان نوشت: سهیل بیرقی در نشست رسانه‌ای بعد ...
15731,3,| قیمت هر متر مربع آپارتمان مسکونی در منطقه ۱...
21241,0,سرپرست جمعیت هلال احمر شهرستان مرند گفت: تلاش ...


We shuffle the dataset to add an element of randomness .

In [52]:
from sklearn.utils import shuffle

In [53]:
df= shuffle(df)

In [54]:
df.head()

,labels,text
26894,2,عصر هفتم اردیبهشت در مراسمی که با حضور جمع زیا...
27013,2,فرهاد عزیزی مدیرکل امور پایگاههای میراث جهانی ...
3532,3,مدیرعامل شرکت توزیع نیروی برق استان کرمانشاه ا...
26831,2,| همانطور که پیش از این در خبرها به آن پرداخت...
4254,1,| مرجع تقلید شیعه در قم گفت: روز 9 دی، روزی ب...


We split the data so that we have 9000 news files as traing data and the rest is for validation .

In [55]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

In [56]:
n_trn = 9000
df_train, df_valid = split_vals(df, n_trn)

In [57]:
df_valid.shape

(1171, 2)

In [58]:
df_train.head()

,labels,text
26894,2,عصر هفتم اردیبهشت در مراسمی که با حضور جمع زیا...
27013,2,فرهاد عزیزی مدیرکل امور پایگاههای میراث جهانی ...
3532,3,مدیرعامل شرکت توزیع نیروی برق استان کرمانشاه ا...
26831,2,| همانطور که پیش از این در خبرها به آن پرداخت...
4254,1,| مرجع تقلید شیعه در قم گفت: روز 9 دی، روزی ب...


We have 4 labels referring to 4 news categories . 

In [59]:
set(df_train["labels"])

{0, 1, 2, 3}

As before we save our data and then tokenize it .

In [60]:
df_train.to_pickle('Clas_train_df.pkl')
df_valid.to_pickle('Clas_valid_df.pkl')

In [61]:
df_train.to_csv('Clas_train.csv', header=False, index=False)
df_valid.to_csv('Clas_valid.csv', header=False, index=False)

In [62]:
df_train.text[1]

'\xa0معاون بازاريابي فروش ايران خودرو با بيان اين كه اجراي روش هاي متنوع و جامع براي فروش محصولات يكي از مسيرهاي افزايش رضايتمندي مشتريان و فراهم كردن امكان خريد كالاي ايراني است تاكيد كرد: ايران خودرو در سال حمايت از كالاي ايراني برنامه هاي متنوعي را براي خريد محصولات به اجرا خواهد گذاشت. \xa0\xa0 \xa0\xa0به گزارش ايكوپرس، مصطفي خان كرمي با تشريح برنامه هاي فروش ايران خودرو در سال 97، اعمال تخفيف خريد كالاي ايراني را از جمله مشوق هاي خريد محصولات ايران خودرو دانست و اظهار كرد:\u200c با مطالعه و بررسي بازار و نيازسنجي از مشتريان برنامه هاي فروش را منطبق با خواست و سليقه آنان به اجرا مي گذاريم.وي افزود:\u200c طرح هاي فروش را متناسب با بودجه مشتريان در نظر مي گيريم و در اين راستا برنامه هاي فروش اقساطي و اعتباري را طرح ريزي مي كنيم تا همه گروه هاي درآمدي موفق به خريد محصولات ايراني ايران خودرو شوند.خان كرمي طرح هاي پيش فروش را از جمله طرح هاي فروش محصولات ايران خودرو نام برد و گفت: در پيش فروش هايی كه به صورت سرمايه گذاري پيشنهاد مي شود امكان مشاركت مشتريان در فرآيند توليد محصول ايجاد مي

In [63]:
chunksize=1000

In [64]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace(':\u200c', " ").replace('\xa0',' ').replace('\r\n',' ').replace('|',' ').replace('«', '.')
    return re1.sub(' ', html.unescape(x))

In [65]:
def get_texts(df, n_lbls=1):
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
    for i in range(n_lbls+1, len(df.columns)): texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)
    texts = list(texts.apply(fixup).values)

    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok, list(labels)

In [66]:
def get_all(df, n_lbls):
    tok, labels = [], []
    
    for i, r in enumerate(df): 
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        tok += tok_;
        labels += labels_
    return tok, labels

In [67]:
df_trn = pd.read_csv('Clas_train.csv', header=None, chunksize=chunksize)
df_val = pd.read_csv('Clas_valid.csv', header=None, chunksize=chunksize)

In [68]:
tok_trn, trn_labels = get_all(df_trn, 1)
tok_val, val_labels = get_all(df_val, 1)

0


/home/bahram/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/bahram/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


1
2
3
4
5
6
7
8
0
1


Here we just saved the tokens .

In [69]:
(CLAS_PATH/'tmp').mkdir(exist_ok=True)

np.save(CLAS_PATH/'tmp'/'tok_trn.npy', tok_trn)
np.save(CLAS_PATH/'tmp'/'tok_val.npy', tok_val)

np.save(CLAS_PATH/'tmp'/'trn_labels.npy', trn_labels)
np.save(CLAS_PATH/'tmp'/'val_labels.npy', val_labels)

In [70]:
tok_trn = np.load(CLAS_PATH/'tmp'/'tok_trn.npy')
tok_val = np.load(CLAS_PATH/'tmp'/'tok_val.npy')

Now we load itos we saved before in languge model part and make a stoi function from it .

In [71]:
itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

45303

Classifier !

In [72]:
trn_clas = np.array([[stoi[o] for o in p] for p in tok_trn])
val_clas = np.array([[stoi[o] for o in p] for p in tok_val])

In [73]:
np.save(CLAS_PATH/'tmp'/'trn_ids.npy', trn_clas)
np.save(CLAS_PATH/'tmp'/'val_ids.npy', val_clas)

In [74]:
trn_clas = np.load(CLAS_PATH/'tmp'/'trn_ids.npy')
val_clas = np.load(CLAS_PATH/'tmp'/'val_ids.npy')

In [75]:
trn_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'trn_labels.npy'))
val_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'val_labels.npy'))

The hyperparameters should be same as before as we are going to load the encoder . but the dropouts and bptt can be changed .

In [ ]:
bptt,em_sz,nh,nl = 70,400,1150,4
vs = len(itos)
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
bs = 64

In [77]:
min_lbl = trn_labels.min()
trn_labels -= min_lbl
val_labels -= min_lbl
c=int(trn_labels.max())+1

The first thing we need is a Dataset. So we have a Dataset passing in the documents and their labels.then we use sortish which makes it so the first things in the list are, on the whole, shorter than the things at the end, but a little bit random as well.
sampler parameter and sampler is a class we are going to define that tells the data loader how to shuffle. 
For validation set, we are going to define something that actually just sorts. It just deterministically sorts it so that all the shortest documents will be at the start, all the longest documents will be at the end, and that’s going to minimize the amount of padding.


In [78]:
trn_ds = TextDataset(trn_clas, trn_labels)
val_ds = TextDataset(val_clas, val_labels)
trn_samp = SortishSampler(trn_clas, key=lambda x: len(trn_clas[x]), bs=bs//2)
val_samp = SortSampler(val_clas, key=lambda x: len(val_clas[x]))
trn_dl = DataLoader(trn_ds, bs//2, transpose=True, num_workers=1, pad_idx=1, sampler=trn_samp)
val_dl = DataLoader(val_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=val_samp)
md = ModelData(PATH, trn_dl, val_dl)

In [79]:
dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.5


So now we can call that function as before .this timewith the head we add on, we have a few more things we can do. we can add more than one hidden layer. In layers=[em_sz*3, 50, c]
em_sz*3 : is the input to the classifier head 
50: output of the first layer .
c : output of the last layer or the number of classes . 

In [80]:
m = get_rnn_classifier(bptt, 20*70, c, vs, emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
          layers=[em_sz*3, 50, c], drops=[0.4, 0.1],
          dropouti=dps[0], wdrop=dps[1], dropoute=dps[2], dropouth=dps[3])

In [81]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))


We are going to use RNN_Learner just like before.

In [82]:
learn = RNN_Learner(md, TextModel(to_gpu(m)), opt_fn=opt_fn)
learn.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learn.clip=25.
learn.metrics = [accuracy]

In [83]:
lr=3e-3
lrm = 2.6


In [96]:
lrs=np.array([1e-5,1e-4,1e-4,1e-3,1e-2,1e-2])

Here we use weight decay and load the encoder we have saved .

In [97]:
wd = 1e-7

learn.load_encoder('LM_enc')

Now we just train the last layer and we get 89% accuracy !

In [86]:
learn.freeze_to(-1)


In [89]:
learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))

epoch      trn_loss   val_loss   accuracy                    
    0      0.367081   0.870484   0.899231  



[array([0.87048]), 0.8992314260297103]

In [98]:
learn.freeze_to(-2)

In [99]:
learn.fit(lrs, 5, wds=wd, cycle_len=1, use_clr=(8,3))

epoch      trn_loss   val_loss   accuracy                    
    0      0.302214   131.898343 0.903501  
    1      0.247385   0.321458   0.911187                    
    2      0.205509   14.199218  0.920581                    
    3      0.211435   7.438195   0.923997                    
    4      0.245716   4.298051   0.923143                    



[array([4.29805]), 0.9231426131766032]

In [93]:
learn.save('Class_unfreeze=2_acc=93')

In [94]:
m= learn.model
m.eval()

SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(45303, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(45303, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
      (3): LockedDropout(
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): ModuleList(
      (0): LinearBlock(
        (lin): Linear(in_features=1200, out_features=50, bias=True)
        (drop): Dropout(p=0.2)
        (bn): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True)
      )
      (1): LinearBlock(
        (lin): Linea

In [25]:
learn.freeze_to(-3)

In [26]:
learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))

epoch      trn_loss   val_loss   accuracy                    
    0      0.297732   0.232494   0.918873  



[array([0.23249]), 0.9188727579190118]